## Introdução

Este arquivo irá aplicar a ideia desta api que é retornar a quantidade de calorias com base nas seguintes entradas:
- Nome do(s) Alimento(s) (Obrigatório);
- Porção(ões) de(os) alimento(s) em gramas (Não obrigatório, 100g por padrão)

Pensando nisso, os itens que serão testados são:
- O nome do alimento e como ele será filtrado;
- O calculo das calorias dos alimento numa porção de 100g para cada alimento;
- O calculo das calorias dos alimentos em diferentes porções;
- A resposta da API


## Importando Bibliotecas

In [1]:
import pandas as pd
import unicodedata
import re

## Coletando os dados

In [2]:
url = 'https://raw.githubusercontent.com/LucaseNogueira/Caloric_Amount_Food_Group/master/data/Taco_Calorias_Alimentos.csv'
df = pd.read_csv(url)
df

,Código do alimento,Descrição do alimento,Calorias
0,6300101,"Arroz (polido, parboilizado, agulha, agulhinha...",135.622946
1,6300201,Arroz integral,130.945644
2,6300701,Milho (em grão),160.141000
3,6300706,Canjiquinha de milho em grão,79.680000
4,6300707,Xerém de milho,62.952000
...,...,...,...
1116,8579003,Maria izabel,154.000000
1117,8579004,Arroz de leite,142.422000
1118,8579005,Arroz com mandioca,130.311473
1119,8579006,Arroz com ovo,179.106973


## Fazendo um teste 1.0

In [3]:
series_teste = pd.Series(['arroz', 'arroz branco', 'arroz integral', 'farinha de trigo', 'farinha de mandioca', 'Xerem de milho', 'milho em grao', 'mandioca'])

In [4]:
series_teste.loc[series_teste.str.startswith('farinha de', na=False)]

3       farinha de trigo
4    farinha de mandioca
dtype: object

In [5]:
series_teste.loc[series_teste.str.startswith('farinha mandioca', na=False)]

Series([], dtype: object)

In [6]:
series_teste.loc[series_teste.str.contains('farinha mandioca', regex=False)]

Series([], dtype: object)

In [7]:
series_teste.loc[series_teste.str.contains('farinha|mandioca')]

3       farinha de trigo
4    farinha de mandioca
7               mandioca
dtype: object

In [8]:
series_teste.loc[series_teste.str.contains('mandioca')]

4    farinha de mandioca
7               mandioca
dtype: object

In [9]:
series_teste.loc[series_teste.str.contains('farinha.*mandioca.*')]

4    farinha de mandioca
dtype: object

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def sanitizar(frase:str)->str:
  sw = stopwords.words('portuguese')
  str_processada = re.sub(r"[,.()]", "", frase)
  str_processada = str_processada.replace('etc','')
  str_processada = ' '.join([k for k in str_processada.split(" ") if k not in sw])
  str_processada = str_processada.upper()
  str_processada = unicodedata.normalize("NFD", str_processada)
  str_processada = str_processada.encode("ascii", "ignore").decode("utf-8").strip()
  str_processada = re.sub(r"[^A-Z0-9]", " ", str_processada)
  return str_processada

In [12]:
original = " peça ótimo, péssimo não é tão às (Algo assim que devemos filtrar). "
#sw = stopwords.words('portuguese')

# Removendo acentos com unicodedata
#processamento = ' '.join([k for k in original.split(" ") if k not in sw])
#processamento = processamento.upper()
#processamento = unicodedata.normalize("NFD", processamento)
#processamento = processamento.encode("ascii", "ignore").decode("utf-8").strip()
#processamento = re.sub(r"[,.()]", "", processamento)
#processamento = re.sub(r"[^A-Z0-9]", " ", processamento)
processamento = sanitizar(original)
processamento

'PECA OTIMO PESSIMO TAO ALGO ASSIM DEVEMOS FILTRAR'

## DEV Complementar: Adicionando coluna Campo de Pesquisa no dataframe

Aqui será criado uma coluna responsável apenas para facilitar a pesquisa dos alimentos. Esta coluna contem os dados de **Descrição do Alimento** sanetizado, sem caracterez especiais e padronizados.

**Esta documentado pois este desenvolvimento foi migrado para o arquivo FoodCaloriesTransfer**

In [13]:
#coluna_descricao_tratada = df['Descrição do alimento'].str.strip()
#coluna_campo_pesquisa = []

#for col_str in coluna_descricao_tratada:
#  str_processada = sanitizar(col_str)
#  coluna_campo_pesquisa.append(str_processada)

#df['Campo de Pesquisa'] = coluna_campo_pesquisa
#df

## O nome do alimento e como ele será filtrado

Aqui temos o seguinte desafio:
- O usuário pode não ter ciência do nome completo de um alimento na tabela Taco, por exemplo, na tabela Taco existem tipos diferentes de arroz, porém para o usuário é tudo a mesma coisa.
  - Desta forma nós vamos retornar a primeira ocorrência dos alimentos, ordenando pelo código.
- O usuário também pode informar parte de uma palavra e não o nome de um alimento. Usando o arroz como exemplo, o usuário pode informar apenas a letra "a" ou quem sabe apenas as letras "arr".
  - Isso eu não sei como solucionar kkkkkk.
- Devemos filtrar os alimentos sem considerar a pontoação nas palavras.

In [14]:
def filtro(frase:str):
  frase = sanitizar(frase)
  print(len(frase.split(' ')))
  if len(frase.split(' ')) == 1:
    return df.loc[df['Campo de Pesquisa'].str.startswith(frase, na=False)].head(1)
  else:
    str_regex = '^'
    for str_aux in frase.split(' '):
      str_regex += str_aux + '.*'
    return df.loc[df['Campo de Pesquisa'].str.contains(str_regex)].head(1)

In [15]:
str_composto = 'Feijão Preto'
str_simples = 'Milho'

print(filtro(str_composto))
print('-'*90)
print(filtro(str_simples))

2


KeyError: 'Campo de Pesquisa'

## Porções do alimento

Este desafio será mais simples. A medida de todos os alimentos presente na fonte de dados é de **100g**, ou seja, se um alimento consta com **90kcal** significa que são **90kcal** para cada **100g** deste alimento. Aqui queremos dar ao usuário o poder de alterar estas porções. Sendo assim foi elaborado a sequinte formula.

**kcal_estimada = kcal_100g * (porcao_indicada/100)**

A formula apresentada possui viés matematico e não considera nenhum fenomeno fisico ou quimico do alimento ou condições do ambiente.

In [ ]:
def caloria_por_porcao(porcao:float, df)->float:
  return df['Calorias'] * (porcao/100)

In [ ]:
str_lasanha = 'Lasanha'
porcao = 240
df_lasanha = filtro(str_lasanha)
df_lasanha

In [ ]:
print(caloria_por_porcao(porcao,df_lasanha))